# Interaction with the World Homework (#3)
Python Computing for Data Science (c) J Bloom, UC Berkeley 2018

Due Tuesday 2pm, Feb 20, 2018

# 1) Monty: The Python Siri

Let's make a Siri-like program (call it Monty!) with the following properties:
   - record your voice command
   - use a webservice to parse that sound file into text
   - based on what the text, take three different types of actions:
       - send an email to yourself
       - do some math
       - tell a joke

So for example, if you say "Monty: email me with subject hello and body goodbye", it will email you with the appropriate subject and body. If you say "Monty: tell me a joke" then it will go to the web and find a joke and print it for you. If you say, "Monty: calculate two times three" it should response with printing the number 6.

Hint: you can use speed-to-text apps like Houndify (or, e.g., Google Speech https://cloud.google.com/speech/) to return the text (but not do the actions). You'll need to sign up for a free API and then follow documentation instructions for using the service within Python. 

In [ ]:
import os, io
import pyaudio, wave

In [ ]:
WAVE_OUTPUT_FILENAME = "new2.wav"
chunk = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 5

def record(WAVE_OUTPUT_FILENAME = "new2.wav", chunk = 1024, 
       FORMAT = pyaudio.paInt16, CHANNELS = 1, RATE = 44100, RECORD_SECONDS = 5):
    p = pyaudio.PyAudio()
    stream = p.open(format = FORMAT, channels = CHANNELS,
                    rate = RATE, input = True,
                    frames_per_buffer = chunk)
    all = []
    for i in range(0, int(RATE / chunk * RECORD_SECONDS)):
        data = stream.read(chunk)
        all.append(data)
    print("* done recording")
    stream.close()
    p.terminate()
    data = b"".join(all)
    with wave.open(WAVE_OUTPUT_FILENAME, "wb") as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(data)

In [ ]:
record()

In [ ]:
!ls -lat |head

In [ ]:
!open new2.wav

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/smerdis/service-account-file.json'

In [ ]:
def transcribe_file(speech_file):
    """Transcribe the given audio file."""
    from google.cloud import speech
    from google.cloud.speech import enums
    from google.cloud.speech import types
    client = speech.SpeechClient()

    with io.open(speech_file, 'rb') as audio_file:
        content = audio_file.read()

    audio = types.RecognitionAudio(content=content)
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=44100,
        language_code='en-US')

    response = client.recognize(config, audio)
    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        print('Transcript: {}'.format(result.alternatives[0].transcript))
        
    return response.results

In [ ]:
results = transcribe_file(WAVE_OUTPUT_FILENAME)

In [ ]:
command = results[0].alternatives[0].transcript

In [ ]:
print(command)

In [ ]:
def tell_joke():
    """Function that gets a joke off the internet and returns it."""
    raise NotImplementedError("TODO")
    
def do_calculation(command):
    """Function that accepts a command to do math and returns the result.
    
    Probably just gonna throw the calculation at google and try to return whatever is in the calculator box."""
    from urllib.parse import urlencode
    import bs4
    import requests
    
    words = command.split()
    the_calculation = ' '.join(words[2:])
    print(the_calculation)
    
    query = {'q':the_calculation}
    url_values = urlencode(query)
    
    url = f"https://www.google.com/search?{url_values}"
    req = requests.get(url)
    #print(req.text)
    soup = bs4.BeautifulSoup(req.text, "lxml")
    print(list(x.get_text() for x in soup.find_all("h2", class_="r")))
    
def send_email(command):
    """Function that accepts a command to send an email with a specifiable subject and body, then does so."""
    raise NotImplementedError("TODO")

def act(command):
    """Function that takes the transcript of a command, determines if it is valid, and acts accordingly if so."""
    words = command.split()
    # I decided to just use a simple heuristic to dispatch the 3 actions specified in the problem
    # rather than getting into natural language processing and other heavyweight approaches
    if (words[0] != "Monty") and (words[0] != "Monte"):
        print("Commands must begin by addresing Monty by name. For example: \"Monty, tell me a joke.\"")
    elif "email" in command: # if we see the word email, assume we are to send one
        # putting this first allows the email to contain the words "calculate" or "tell me a joke"...
        print("Gonna send you an email, aw yiss.")
        send_email(command)
    elif "calculate" in command:
        print("Gonna do some math, aw yiss.")
        do_calculation(command)
    elif ("tell" in command and "joke" in command):
        print("Gonna tell you  a joke, aw yiss.")
        tell_joke()

In [ ]:
act(command)

# 2) Write a program that identifies musical notes from sound (AIFF) files. 

  - Run it on the supplied sound files (12) and report your program’s results. 
  - Use the labeled sounds (4) to make sure it works correctly. The provided sound files contain 1-3 simultaneous notes from different organs.
  - Save copies of any example plots to illustrate how your program works.
  
  https://piazza.com/berkeley/spring2018/ay250class13410/resources -> Homeworks -> hw3_sound_files.zip

Hints: You’ll want to decompose the sound into a frequency power spectrum. Use a Fast Fourier Transform. Be care about “unpacking” the string hexcode into python data structures. The sound files use 32 bit data. Play around with what happens when you convert the string data to other integer sizes, or signed vs unsigned integers. Also, beware of harmonics.